In [1]:
import nltk, string, gensim


In [2]:
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Error loading abc: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>
[nltk_data] Downloading package punkt to /home/dai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk.corpus import abc

In [4]:
abc.sents()

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ...]

In [5]:
model= gensim.models.Word2Vec(abc.sents())

In [6]:
data=model.wv.most_similar('science')
print(data)

[('law', 0.9348715543746948), ('policy', 0.9129614233970642), ('general', 0.9111557602882385), ('agriculture', 0.9049667716026306), ('discussion', 0.9038485288619995), ('education', 0.9037929177284241), ('media', 0.9018175601959229), ('biology', 0.8981183171272278), ('department', 0.8980792164802551), ('physics', 0.8975552916526794)]


In [7]:
import json
json_file='intents.json'
with open('intents.json','r') as f:
  data= json.load(f)

In [8]:
data

[{'tag': 'welcome',
  'patterns': ['Hi',
   'How are you',
   'Is any one to talk?',
   'Hello',
   'hi are you available'],
  'responses': ['Hello, thanks for contacting us',
   'Good to see you here',
   ' Hi there, how may I assist you?']},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye', 'I will come back soon'],
  'responses': ['See you later, thanks for visiting',
   'have a great day ahead',
   'Wish you Come back again soon.']},
 {'tag': 'thankful',
  'patterns': ['Thanks for helping me',
   'Thank your guidance',
   "That's helpful and kind from you"],
  'responses': ['Happy to help!',
   'Any time!',
   'My pleasure',
   'It is my duty to help you']},
 {'tag': 'hoursopening',
  'patterns': ['What hours are you open?',
   'Tell your opening time?',
   'When are you open?',
   'Just your timing please'],
  'responses': ["We're open every day 8am-7pm",
   'Our office hours are 8am-7pm every day',
   'We open office at 8 am and close at 7 pm']},
 {'tag': 'pay

In [9]:
import pandas as pd
df= pd.DataFrame(data)


In [10]:
df['patterns']=df['patterns'].apply(','.join)

In [11]:
df

,tag,patterns,responses
0,welcome,"Hi,How are you,Is any one to talk?,Hello,hi ar...","[Hello, thanks for contacting us, Good to see ..."
1,goodbye,"Bye,See you later,Goodbye,I will come back soon","[See you later, thanks for visiting, have a gr..."
2,thankful,"Thanks for helping me,Thank your guidance,That...","[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,"What hours are you open?,Tell your opening tim...","[We're open every day 8am-7pm, Our office hour..."
4,payments,"Can I pay using credit card?, Can I pay using ...","[We accept VISA, Mastercard and credit card, W..."


In [12]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [13]:
from nltk.tokenize import word_tokenize

In [14]:
from nltk.corpus import stopwords
from textblob import Word
stop=stopwords.words('english')
df['patterns']=df['patterns'].apply(lambda x:' '.join(x.lower() for x in x.split()))
df['patterns']=df['patterns'].apply(lambda x:' '.join(x for x in word_tokenize(x) if x not in string.punctuation))
df['patterns']=df['patterns'].str.replace('[^\W\S]','')
df['patterns']=df['patterns'].apply(lambda x:' '.join(x for x in x.split() if not x.isdigit()))
df['patterns']=df['patterns'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))
df['patterns']=df['patterns'].apply(lambda x:' '.join([Word(word).lemmatize() for word in x.split()]))

In [15]:
df

,tag,patterns,responses
0,welcome,hi one talk hello hi available,"[Hello, thanks for contacting us, Good to see ..."
1,goodbye,bye see later goodbye come back soon,"[See you later, thanks for visiting, have a gr..."
2,thankful,thanks helping thank guidance 's helpful kind,"[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,hour open tell opening time open timing please,"[We're open every day 8am-7pm, Our office hour..."
4,payments,pay using credit card pay using mastercard pay...,"[We accept VISA, Mastercard and credit card, W..."


In [16]:
from gensim.models import Word2Vec

In [17]:
# taking the outer list
bigger_list=[]
for i in df['patterns']:
    li=i.split()
    bigger_list.append(li)

In [18]:
bigger_list

[['hi', 'one', 'talk', 'hello', 'hi', 'available'],
 ['bye', 'see', 'later', 'goodbye', 'come', 'back', 'soon'],
 ['thanks', 'helping', 'thank', 'guidance', "'s", 'helpful', 'kind'],
 ['hour', 'open', 'tell', 'opening', 'time', 'open', 'timing', 'please'],
 ['pay',
  'using',
  'credit',
  'card',
  'pay',
  'using',
  'mastercard',
  'pay',
  'using',
  'cash']]

In [19]:
#custom data is fed to machine for further processing
model=Word2Vec(bigger_list,min_count=1,workers=4)  # workers -> Multiple threads
print(model)

Word2Vec<vocab=32, vector_size=100, alpha=0.025>


In [20]:
# To save the model and we can transfer this files(word2vec.model,model.bin) and load model in another pc
model.save("word2vec.model")
model.save("model.bin")

In [21]:
new_model=Word2Vec.load('model.bin')  # This code load the model

In [22]:
vocab=list(new_model.wv.key_to_index)
vocab

['using',
 'pay',
 'hi',
 'open',
 'later',
 'soon',
 'back',
 'come',
 'goodbye',
 'bye',
 'see',
 'helping',
 'available',
 'hello',
 'talk',
 'one',
 'thanks',
 'cash',
 'thank',
 'mastercard',
 "'s",
 'helpful',
 'kind',
 'hour',
 'tell',
 'opening',
 'time',
 'timing',
 'please',
 'credit',
 'card',
 'guidance']

In [23]:
similar_words=new_model.wv.most_similar('kind')
print(similar_words)

[('one', 0.19613029062747955), ('pay', 0.18879325687885284), ('guidance', 0.14262472093105316), ('credit', 0.13661333918571472), ('hour', 0.10765138268470764), ('see', 0.09932280331850052), ('bye', 0.0777018591761589), ('opening', 0.075437992811203), ('helpful', 0.06751954555511475), ('mastercard', 0.049430545419454575)]


In [24]:
similar_words=new_model.wv.most_similar('time')
print(similar_words)

[('later', 0.20086249709129333), ('back', 0.177496075630188), ('please', 0.1668374240398407), ('opening', 0.11890643835067749), ('mastercard', 0.11017192900180817), ('hour', 0.10137568414211273), ('come', 0.09656082838773727), ('bye', 0.08370571583509445), ('helpful', 0.08267958462238312), ('cash', 0.0762253850698471)]


In [25]:
similar_words=new_model.wv.most_similar('bye')
print(similar_words)

[('using', 0.21609024703502655), ('please', 0.12639683485031128), ('credit', 0.10308832675218582), ('time', 0.08370570838451385), ('kind', 0.07770185172557831), ('guidance', 0.06291314214468002), ('goodbye', 0.04467068612575531), ('card', 0.03011849895119667), ('hour', 0.029784908518195152), ('thank', 0.020197447389364243)]


In [26]:
df['responses'][3][2]

'We open office at 8 am and close at 7 pm'

In [27]:
dissimilar_words=model.wv.doesnt_match('''see you later, thanks for visiting'''.split())
print(dissimilar_words)

see


In [28]:
df

,tag,patterns,responses
0,welcome,hi one talk hello hi available,"[Hello, thanks for contacting us, Good to see ..."
1,goodbye,bye see later goodbye come back soon,"[See you later, thanks for visiting, have a gr..."
2,thankful,thanks helping thank guidance 's helpful kind,"[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,hour open tell opening time open timing please,"[We're open every day 8am-7pm, Our office hour..."
4,payments,pay using credit card pay using mastercard pay...,"[We accept VISA, Mastercard and credit card, W..."
